<a href="https://colab.research.google.com/github/ManJ-PC/Psychosis-AI/blob/master/workshop_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>web scraping <small>(em python)</small></h1>
<p>um caso de estudo no Sigarra</p>

_27/05/2021_

<h3 style="font-weight: 300;">
    tarefa de extrair informação de websites através do protocolo HTTP
</h3>

<h3 style="font-weight: 300;">
    pode ser manual, mas normalmente refere-se à extração automatizada através de <i>bots/crawlers</i>
</h3>

## quando usar?

1. recolha de dados
    1. não há uma web API
    1. há uma web API com rate limits
1. monitorizar o estado de um website ou processo
1. automatizar ações humanas
1. mapear as páginas de um website (sitemap)
1. testar interfaces

## conhecimentos necessários
1. HTML
1. seletores [CSS](https://www.w3schools.com/cssref/css_selectors.asp)

## conhecimentos úteis
1. seletores [XPATH](https://www.w3schools.com/xml/xpath_syntax.asp)
1. browser developer tools
1. protocolo HTTP (`GET` vs `POST`, headers, cookies, HTTP errors)
1. JavaScript
1. Regular Expressions

## pitão e mais o quê?
* [requests](https://pypi.org/project/requests/) - cliente HTTP | [docs](https://docs.python-requests.org/en/master/)
    * ou [urllib3](https://pypi.org/project/urllib3/) - outro cliente HTTP | [docs](https://urllib3.readthedocs.io/en/latest/)
* [beautifulsoup4](https://pypi.org/project/beautifulsoup4/) - HTML/XML parsing utilities | [docs](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
    * ou [lxml](https://pypi.org/project/lxml/) - outra biblioteca para HTML/XML parsing utilities | [docs](https://lxml.de/)
* [scrapy](https://pypi.org/project/Scrapy/) - web crawling framework | [docs](https://docs.scrapy.org/en/latest/)
* [selenium](https://pypi.org/project/selenium/) - automatizar interações com browsers | [docs](https://www.selenium.dev/documentation/en/)
* [selenium driver](https://selenium-python.readthedocs.io/installation.html#drivers) específico para o browser ([chrome](https://sites.google.com/a/chromium.org/chromedriver/downloads), [firefox](https://github.com/mozilla/geckodriver/releases), ...)
    * [ghostdriver](https://github.com/detro/ghostdriver) parecido com selenium mas especialmente feito para [PhantomJs](https://phantomjs.org/)

In [ ]:
!pip3 install urllib3 requests beautifulsoup4 Scrapy selenium > /dev/null

## `requests`
#### Cliente HTTP

[`pip install requests`](https://pypi.org/project/requests/) | [docs](https://docs.python-requests.org/en/master/)

In [ ]:
!pip3 install requests > /dev/null

In [ ]:
import requests

In [ ]:
feup = requests.get("https://sigarra.up.pt/feup/pt")
print(feup.text)

`requests` - propriedades das respostas 

In [ ]:
# feup - compiladores
comp = "https://sigarra.up.pt/feup/pt/ucurr_geral.ficha_uc_view?pv_ocorrencia_id=272732"
# github api
github = "https://api.github.com/user"

In [ ]:
for site in [comp, github]:
    r = requests.get(site)
    print(site)
    print("* HTTP status     = %s" % r.status_code)
    print("* HTTP CT headers = %s" % r.headers['content-type'])
    print("* encoding        = %s" % r.encoding)
    print("* text            = %s" % r.text[0:100])
    print("---\n")

`requests` - passagem de parâmetros

In [ ]:
payload = {'pv_ocorrencia_id': 272732} # 272832
r = requests.get("https://sigarra.up.pt/feup/pt/ucurr_geral.ficha_uc_view", params=payload)
print(r.url)
print(r.text[3300:3360])

## `urllib`
#### Cliente HTTP

[`pip install urllib3`](https://pypi.org/project/urllib3/) | [docs](https://urllib3.readthedocs.io/en/latest/)

In [ ]:
!pip3 install urllib3 > /dev/null

In [ ]:
from urllib.request import urlopen

In [ ]:
# feup - compiladores
comp = "https://sigarra.up.pt/feup/pt/ucurr_geral.ficha_uc_view?pv_ocorrencia_id=272732"

In [ ]:
r = urlopen(comp)
print("HTTP status code = %s" % r.status)
html = r.read()
print(html[3300:3360])

## `beautifulsoup4`
#### HTML/XML parsing utilities

[`pip install beautifulsoup4`](https://pypi.org/project/beautifulsoup4/) | [docs](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [ ]:
!pip3 install beautifulsoup4 > /dev/null

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup("<p>Some<b>bad<i>HTML")
print(soup.prettify())

In [ ]:
r = requests.get("https://sigarra.up.pt/feup/pt/ucurr_geral.ficha_uc_view", params={'pv_ocorrencia_id': 272732})
print(r.status_code, r.url)

In [ ]:
soup = BeautifulSoup(r.text)

In [ ]:
print(soup.prettify()[0:100])

#### navegar pelo HTML - acesso por atributos
retorna sempre a primeira ocorrência

In [ ]:
soup.title

In [ ]:
print(soup.title.name, ":", soup.title.string)

In [ ]:
# soup.p.a.next_sibling.attrs["href"] #.get("href")
# print(soup.p)
print(soup.p.prettify())

como extrair código e sigla?

```html
<table class="formulario">
    <tr>
        <td class="formulario-legenda">Código:</td>
        <td>EIC0028</td>
        <td>&nbsp;&nbsp;&nbsp;</td>
        <td class="formulario-legenda">Sigla:</td>
        <td>COMP</td>
    </tr>
</table>
```

In [ ]:
soup.table.td#.next_sibling#.next_sibling

In [ ]:
soup.select_one("table.formulario td:nth-child(2)")#.string

In [ ]:
soup.select_one("table.formulario td:not(.formulario-legenda)").text

`string` vs `text` vs `stripped_strings`


```html
<table class="formulario">
    <tr>
        <td class="formulario-legenda">Código:</td>
        <td>EIC0028</td>
        <td>&nbsp;&nbsp;&nbsp;</td>
        <td class="formulario-legenda">Sigla:</td>
        <td>COMP</td>
    </tr>
</table>
```

In [ ]:
print("*** table.string      : '%s'" % soup.table.string)
print("*** table.text        : '%s'" % soup.table.text)
print("*** td.string         : '%s'" % soup.table.td.string)
print("*** td.text           : '%s'" % soup.table.td.text)
print("*** td.get_text       : '%s'" % soup.table.td.get_text(strip=True))
print("*** table.get_text    : '%s'" % soup.table.get_text("|", strip=True))
print("*** stripped_strings  : '%s'" % list(soup.table.stripped_strings))

### `select` (css) vs `find_all` (object search)

In [ ]:
r = requests.get("https://sigarra.up.pt/feup/pt/ucurr_geral.ficha_uc_view", params={'pv_ocorrencia_id': 272732})
soup = BeautifulSoup(r.text)

In [ ]:
len(soup.select("table")), len(soup.find_all("table"))

In [ ]:
len(soup.select("table.formulario")), len(soup.find_all("table.formulario"))

In [ ]:
len(soup.select("table.formulario")), len(soup.find_all("table", attrs={"class": "formulario"}))

In [ ]:
import re # regular expressions
len(soup.find_all(re.compile("^tab.e$")))

In [ ]:
def has_class_and_id(tag): 
    return tag.has_attr('class') and tag.has_attr('id')
soup.find_all(has_class_and_id)#[0].a.get("href")

Extrair todos os `href` disponíveis a partir dos `<a href="...">links</a>`

In [ ]:
hrefs = [a.get('href') for a in soup.find_all("a") if a.get('href')] # ou a.attrs["href"]
print(len(hrefs), hrefs[0]) 

In [ ]:
import urllib.parse
base = "https://sigarra.up.pt/feup/pt/ucurr_geral.ficha_uc_view"

In [ ]:
hrefs = [urllib.parse.urljoin(base, h) for h in hrefs]
print(len(hrefs), hrefs[0])

In [ ]:
for h in hrefs[1:9]:
    print(h, end="")
    r = requests.get(h)
    if r.status_code == 200: print(" ", BeautifulSoup(r.text).title.text)
    else:                    print(" ", r)

Dada a página de um professor, como descobrir outros? 

paiva cardoso: https://sigarra.up.pt/feup/pt/func_geral.formview?p_codigo=449856

Como investigar o que é o `p_codigo`? 

In [ ]:
for i in range(449856, 449868):
    r = requests.get("https://sigarra.up.pt/feup/pt/func_geral.formview", params={'p_codigo': i})
    soup = BeautifulSoup(r.text)
    print(i, soup.title.text)

In [ ]:
for i in range(500045, 500065):
    r = requests.get("https://sigarra.up.pt/feup/pt/func_geral.formview", params={'p_codigo': i})
    soup = BeautifulSoup(r.text)
    if soup.title.get_text(strip=True) != "FEUP - Registo não encontrado":
        print(i, soup.title.text)

## `Scrapy`
#### web crawling framework

tem métodos de pesquisa diferentes de Beautiful Soup mas fáceis de perceber: 

`tag.css()` e `tag.xpath()`

[`pip install Scrapy`](https://pypi.org/project/Scrapy/) | [docs](https://docs.scrapy.org/en/latest/)

In [ ]:
!pip3 install Scrapy > /dev/null

In [ ]:
!scrapy startproject sigarraSpider > /dev/null

<img style="height:350px;float:right" src="https://user-images.githubusercontent.com/70276378/119811145-0e34db80-bee7-11eb-9d11-83145645df50.png"></img>

* `scrapy.cfg` and `settings.py` - configurations see [full list](https://docs.scrapy.org/en/latest/topics/settings.html#topics-settings-ref)
* `items.py` - our custom object Models
* `spiders/` - our crawling rules - return dict or Model
* `pipelines.py` - how to post-process collected data
* `middlewares.py` - customize how Scrapy fetches pages

You can start your first spider with:
* `cd sigarraSpider`
* `scrapy genspider example example.com`

vamos usar para [esta página](https://sigarra.up.pt/feup/pt/func_geral.querylist?p_estado=ACT&p_amo_id=&p_unidade=151)

In [ ]:
! cd sigarraSpider && scrapy genspider prof \
    "https://sigarra.up.pt/feup/pt/func_geral.querylist?p_estado=ACT&p_amo_id=&p_unidade=151"

prof scraper auto-gerado

<img style="NOTwidth:500px;" src="https://user-images.githubusercontent.com/70276378/119811107-037a4680-bee7-11eb-850d-c63196a5a549.png"/>

```python
import scrapy

class ProfSpider(scrapy.Spider):
    name = 'prof'
    # remover ou generalizar allowed_domains
    allowed_domains = ['url']
    start_urls = ['url']

    def parse(self, response):
        pass # add the magic here

```

<small>
url -> <code>https://sigarra.up.pt/feup/pt/func_geral.querylist?p_estado=ACT&p_amo_id=&p_unidade=151</code>
</small>


prof scraper - recolher nome de cada professor na lista
```python
class ProfSpider(scrapy.Spider):
    name = 'prof'
    start_urls = ['https://sigarra.up.pt/feup/pt/func_geral.querylist?p_estado=ACT&p_amo_id=&p_unidade=151']

    def parse(self, response):
        for prof in response.css('#conteudoinner ul>li'):
            yield {'name': prof.css('a ::text').get()}
```

In [ ]:
!cd sigarraSpider && scrapy crawl prof -o result.json -t json

Prof Scraper - para cada professor da lista, ir à respetiva página e recolher mais dados

Adicionar o `yield from`
```python
    def parse(self, response):
        for prof in response.css('#conteudoinner ul>li'):
            yield {'name': prof.css('a ::text').get()}

        yield from response.follow_all(css='#conteudoinner ul>li>a', callback=self.parse_prof)
```
criar o novo método:
```python
    def parse_prof(self, response):
        email = response.xpath("//td[text()='Email Institucional:']//following-sibling::td//text()").get()
        if email: email+="@fe.up.pt"
        yield {
            'nome': response.xpath("//td[text()='Nome:']//following-sibling::td/b/text()").get(),
            'categoria': response.xpath("//td[text()='Categoria:']//following-sibling::td/text()").get(),
            'email':  email
        }
```

In [ ]:
!cd sigarraSpider && scrapy crawl prof -o result.json -t json

## `selenium`
#### automatizar interações com browsers

[`pip install selenium`](https://pypi.org/project/selenium/) | [docs](https://www.selenium.dev/documentation/en/)

requer [selenium driver](https://selenium-python.readthedocs.io/installation.html#drivers) específico para o browser ([chrome](https://sites.google.com/a/chromium.org/chromedriver/downloads), [firefox](https://github.com/mozilla/geckodriver/releases), ...)
    

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located

In [ ]:
driver = webdriver.Chrome(executable_path="/PATH_TO_CHROMEDRIVER/chromedriver")

In [ ]:
# go to a page
driver.get("https://fe.up.pt")

In [ ]:
# depois, procurar o link 'Pessoal' e clicar-lhe
driver.find_element_by_xpath("//a[@title='Pessoal']").click()

In [ ]:
# procurar um elemento no UI
search_box = driver.find_element(By.NAME, "P_NOME")
# search_box = driver.find_element_by_name("P_NOME")
# search_box = driver.find_element_by_css_selector("input[name='P_NOME']")
# search_box = driver.find_element_by_xpath("//input[@name='P_NOME']")

In [ ]:
search_box.send_keys("algo")

In [ ]:
search_box.clear()
search_box.send_keys("vidal")

In [ ]:
search_box.send_keys(Keys.RETURN)

In [ ]:
wait = WebDriverWait(driver, 5)
nome = wait.until(presence_of_element_located((By.XPATH,
           "//td[text()='Nome:']//following-sibling::td/b")))

In [ ]:
print(nome.get_attribute("textContent"))

In [ ]:
driver.close()

## usar com cuidado...
* se a informação é pública, recolher esses dados não é um problema em si, contudo ir contra os termos de uso, ou mesmo fazendo centenas de pedidos por segundo para chegar a esses dados pode ser considerado uma infração/"ataque" contra o site
* se a informação for privada, é basicamente pirataria com nunaces à volta do tipo de informação recolhida
* é fácil detetar muitos pedidos de um mesmo sítio, é fácil bloquear IPs e banir contas

## dicas e conselhos
1. escolher seletores com menos dependências (`body>div>table>tbody>tr>td>div>span.name` vs `span.name.person`)
1. antever exceções
    1. erros HTTP
    1. perda de internet
    1. HTML inválido (`<div>tags não fechadas`)
    1. estrutura HTML inesperada (elementos que só aparecem ocasionalmente)
1. evitar pedidos duplicados (perda de tempo e risco)
1. por vezes a informação que queremos está em mais do que um sítio de formas diferentes - saber escolher qual usar - às vezes até nos atributos (`<span email="john@example.com">Email: john@example.com<span>` - `span.get("email")` vs `span.get_text()`)
1. cuidado com os encodings (idealmente tudo seria `UTF-8`, mas...)
1. conteúdo escondido/invísivel no texto que é lixo mas ocupa espaço (`<span>conteúdo útil acaba aqui. \x09\x0b\x0d</span>`)
1. adicionar HTTP headers para mascarar comportamento automatizado
1. remote servers para correr os nossos scrapers - mais poder de processamento/paralelização - acesso a outros IPs :)